In [25]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [26]:
rows, cols = 40,15
rewards = np.full ((rows,cols),-100)
Reward_depot_spot= [(18,6),(18,7),(18,8),(19,6),(19,7),(19,8),(20,6),(20,7),(20,8)]
for spot in Reward_depot_spot:
    rewards[spot]= 100

In [27]:

actions = ['up', 'right', 'down', 'left']
Q_table = np.zeros((rows, cols, len(actions)))


In [28]:
rewards[39, 1:14] = -1 # aisles 1
rewards[35, 2:14] = -1 # aisles 2
rewards[32, 2:14] = -1 # aisles 3
rewards[27,4:7] = -1 # aisles 4
rewards[18, 2:6] = -1 # aisles 5
rewards[18, 9:14] = -1 # aisles 6
# now columns the vertical aisles 
rewards[18:40,1] = -1 # aisles 7
rewards[19:32,3] = -1 # aisles 8
rewards[21:32,7] = -1 # aisles 9
rewards[18:40,1] = -1 # aisles 10
rewards[18:40,14] = -1 # aisles 11
aisles = [(33, 9), (34, 9)]  # Define positions for aisles
for aisle in aisles:
    rewards[aisle] = -1  # Set each aisle position to -1

In [29]:
rewards_df = pd.DataFrame(rewards)

In [30]:
# Function to apply background coloring for both -1 and 100
def color_cells(val):
    """
    Colors the background of cells with -1 in red and cells with 100 in green for easier identification.
    """
    if val == -1:
        color = 'red'
    elif val == 100:
        color = 'green'
    else:
        color = 'none'  # Leaves the background unchanged for other cells
    return f'background-color: {color}'

# Apply the background coloring function to the DataFrame
styled_df = rewards_df.style.applymap(color_cells)

styled_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
1,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
2,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
3,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
4,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
5,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
6,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
7,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
8,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
9,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100


In [31]:
def is_terminal_state(current_row_index, current_column_index):
    # Adjust to not immediately consider the start as terminal
    if rewards[current_row_index, current_column_index] == -1:
        return False  # For testing, let's not consider any state terminal
    else:
        return False  # Adjust as needed for your actual terminal state logic

# Adjust get_shortest_path to ensure it can run from your test start without immediate termination


In [32]:
def get_starting_location ():
    current_row_index= np.random.randint(rows)
    current_column_index= np.random.randint(cols)
    while is_terminal_state(current_row_index, current_column_index):
        current_row_index= np.random.randint(rows)
        current_column_index= np.random.randint(cols)
    return current_row_index,current_column_index
        

In [33]:
def get_next_action(current_row_index, current_column_index, epsilon):
  #if a randomly chosen value between 0 and 1 is less than epsilon, 
  #then choose the most promising value from the Q-table for this state.
  if np.random.random() < epsilon:
    return np.argmax(Q_table[current_row_index, current_column_index])
  else: #choose a random action
    return np.random.randint(4)

In [34]:
#define a function that will get the next location based on the chosen action
def get_next_location(current_row_index, current_column_index, action_index):
  new_row_index = current_row_index
  new_column_index = current_column_index
  if actions[action_index] == 'up' and current_row_index > 0:
    new_row_index -= 1
  elif actions[action_index] == 'right' and current_column_index < cols - 1:
    new_column_index += 1
  elif actions[action_index] == 'down' and current_row_index < rows - 1:
    new_row_index += 1
  elif actions[action_index] == 'left' and current_column_index > 0:
    new_column_index -= 1
  return new_row_index, new_column_index

In [35]:
#Define a function that will get the shortest path between any location within the warehouse that 
#the robot is allowed to travel and the item packaging location.
def get_shortest_path(start_row_index, start_column_index):
  #return immediately if this is an invalid starting location
  if is_terminal_state(start_row_index, start_column_index):
    return []
  else: #if this is a 'legal' starting location
    current_row_index, current_column_index = start_row_index, start_column_index
    shortest_path = []
    shortest_path.append([current_row_index, current_column_index])
    #continue moving along the path until we reach the goal (i.e., the item packaging location)
    while not is_terminal_state(current_row_index, current_column_index):
      #get the best action to take
      action_index = get_next_action(current_row_index, current_column_index, 1.)
      #move to the next location on the path, and add the new location to the list
      current_row_index, current_column_index = get_next_location(current_row_index, current_column_index, action_index)
      shortest_path.append([current_row_index, current_column_index])
    return shortest_path

In [36]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from IPython.display import clear_output

# # Initialize variables for plotting
# episode_rewards = []  # To track rewards per episode
# update_frequency = 10  # How often to update the plot

# def plot_live(episode, rewards):
#     clear_output(wait=True)  # Clear the previous plot
#     plt.figure(figsize=(10, 5))
#     plt.plot(rewards, label="Average Reward")
#     plt.xlabel("Episode")
#     plt.ylabel("Average Reward")
#     plt.title(f"Episode: {episode} - Average Reward")
#     plt.legend()
#     plt.grid(True)
#     plt.show()

# # Define your training parameters
# epsilon = 0.9
# discount_factor = 0.9
# learning_rate = 0.9

# # Training loop
# for episode in range(1):
#     row_index, column_index = get_starting_location()
#     total_reward = 0  # Reset total reward for the episode

#     while not is_terminal_state(row_index, column_index):
#         action_index = get_next_action(row_index, column_index, epsilon)
#         old_row_index, old_column_index = row_index, column_index
#         row_index, column_index = get_next_location(row_index, column_index, action_index)
        
#         reward = rewards[row_index, column_index]
#         total_reward += reward  # Accumulate rewards
        
#         old_q_value = Q_table[old_row_index, old_column_index, action_index]
#         temporal_difference = reward + (discount_factor * np.max(Q_table[row_index, column_index])) - old_q_value
#         new_q_value = old_q_value + (learning_rate * temporal_difference)
#         print(new_q_value)
#         Q_table[old_row_index, old_column_index, action_index] = new_q_value

#     # Track rewards
#     episode_rewards.append(total_reward)
    
#     # Update plot
#     if (episode + 1) % update_frequency == 0:
#         avg_rewards = np.mean(episode_rewards[max(0, episode - update_frequency + 1):(episode + 1)])
#         plot_live(episode + 1, episode_rewards)

# print("Training complete!")


In [37]:
# print(get_shortest_path(27, 4))

In [38]:
epsilon = 0.9  # Exploration rate
epsilon_decay = 0.99  # Exploration decay rate
min_epsilon = 0.1  # Minimum exploration rate
learning_rate = 0.9  # Learning rate
discount_factor = 0.9  # Discount factor for future rewards


In [39]:
def is_terminal_state(current_row_index, current_column_index):
    return rewards[current_row_index, current_column_index] == -100


In [40]:
def get_starting_location():
    while True:
        row_index = np.random.randint(rows)
        column_index = np.random.randint(cols)
        if not is_terminal_state(row_index, column_index):
            return row_index, column_index


In [41]:
def get_next_action(current_row_index, current_column_index, epsilon):
    if np.random.random() < epsilon:
        return np.argmax(Q_table[current_row_index, current_column_index])
    else:
        return np.random.randint(len(actions))


In [42]:
def get_next_location(current_row_index, current_column_index, action_index):
    new_row_index, new_column_index = current_row_index, current_column_index
    if actions[action_index] == 'up' and current_row_index > 0:
        new_row_index -= 1
    elif actions[action_index] == 'right' and current_column_index < cols - 1:
        new_column_index += 1
    elif actions[action_index] == 'down' and current_row_index < rows - 1:
        new_row_index += 1
    elif actions[action_index] == 'left' and current_column_index > 0:
        new_column_index -= 1
    return new_row_index, new_column_index


In [43]:
for episode in range(100000):
    row_index, column_index = get_starting_location()
    
    while not is_terminal_state(row_index, column_index):
        action_index = get_next_action(row_index, column_index, epsilon)
        old_row_index, old_column_index = row_index, column_index
        row_index, column_index = get_next_location(row_index, column_index, action_index)
        
        reward = rewards[row_index, column_index]
        old_q_value = Q_table[old_row_index, old_column_index, action_index]
        temporal_difference = reward + (discount_factor * np.max(Q_table[row_index, column_index])) - old_q_value
        Q_table[old_row_index, old_column_index, action_index] = old_q_value + (learning_rate * temporal_difference)
    
    # Decay epsilon
    epsilon = max(min_epsilon, epsilon * epsilon_decay)


In [44]:
Q_table

array([[[   0.        ,    0.        ,    0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    0.        ],
        ...,
        [   0.        ,    0.        ,    0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    0.        ]],

       [[   0.        ,    0.        ,    0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    0.        ],
        ...,
        [   0.        ,    0.        ,    0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    0.        ]],

       [[   0.        ,    0.        ,    0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ,    0. 

In [45]:
def extract_policy(Q_table, rewards):
    policy = np.zeros((rows, cols), dtype=object)  # Initialize policy array
    for i in range(rows):
        for j in range(cols):
            if rewards[i, j] == -100:  # Check for non-actionable state
                policy[i, j] = None  # Indicate no action should be taken
            else:
                action_index = np.argmax(Q_table[i, j])
                policy[i, j] = actions[action_index]
    return policy



In [48]:
def visualize_policy(policy):
    policy_symbols = np.zeros_like(policy, dtype=str)
    action_symbols = {'up': '↑', 'right': '→', 'down': '↓', 'left': '←', None: ' '}  # Include representation for None
    
    for i in range(policy.shape[0]):
        for j in range(policy.shape[1]):
            policy_symbols[i, j] = action_symbols[policy[i, j]]
                
    # Print the policy grid
    for row in policy_symbols:
        print(' '.join(row))



In [49]:
policy = extract_policy(Q_table, rewards)
visualize_policy(policy)



                             
                             
                             
                             
                             
                             
                             
                             
                             
                             
                             
                             
                             
                             
                             
                             
                             
                             
  → → → → → → → ↓ ← ← ← ← ← ←
  ↑   ↑     ↑ ↑ ↑           ↑
  ↑   ↑     ↑ ↑ ↑           ↑
  ↑   ↑       ↑             ↑
  ↑   ↑       ↑             ↑
  ↑   ↑       ↑             ↑
  ↑   ↑       ↑             ↑
  ↑   ↑       ↑             ↑
  ↑   ↑       ↑             ↑
  ↑   → → → → ↑             ↑
  ↑   ↑       ↑             ↑
  ↑   ↑       ↑             ↑
  ↑   ↑       ↑             ↑
  ↑   ↑       ↑             ↑
  → → ↑ → → → ↑ ← ← ← ← ← ← ←
  ↑       